# General Note
In order to guide you through the homework, we put "...COMPLETE HERE..." as placeholder for you to complete the homework.

# Disclaimer
Here, we will perform the training without validation. This is not the correct way to
decide for the best model, but it helps in the visualisation part.  
To be in overfitting is fine with us in this toy problem.  
The goal here is to understand if you are able to implement the two classes and  
see that the results have a meaning.

# Update
Download again the data as the new data are cleaned by the initial part which was
containing also the license of the textual files.

# Word Embeddings
Word embeddings are *dense vectors of real numbers* (they have tipically entries that are non-zero). One dense vector per element in a dictionary.
In Natural Language Processing (NLP) you deal with words.

Thus, you have to represent words in a computer, how to do it?
It is possible to use the ASCII representation of the characters composing the word.

However, this kind of representation does not tell you anything about its meaning, but only about what it is exactly.

Generally you want to obtain a sort of representation of your high-dimensional data to a smaller representation.

You can also usa the so-called ''one-hot encoding'' of the word in a vocabulary. Let's represent the word $w$ in a vocabulary of $V$ words:

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where we have $1$ at the position of $w$.

There are different drawbacks in representing the word like that. It could be an enormous representation, but also it is considering the words as independent. We would like to have a representation of the words that contains a sort of notion of *similarity*.

Let's see why.

Suppose we want to build a language model, and that we have seen the following sentences:


*   the girl ran to the shop
*   the boy ran to the shop
*   the girl bought a flower

in the training data. Then, suppose we see sentences never seen before in our training data

*   the boy bought a flower

Maybe, the language model would tell that it is probable the previous sentence. But what about using the following information we have seen in our training data:
*  we have seen the boy and the girl in the same role in the sentence, thus maybe they share some semantic information
*  we have seen the girl in the same relation of the boy we are seeing now

These two facts shold increase the probability of correctness of the sentences.
We mean this for *semantic similarity*. This is a technique to combat the sparsity of the linguistic data.

This representation however rely on the assumption that words appearning in similar contexts are related to each other semantically.


## Getting Dense Word Embeddings
How could we encode semantic similarity in words? Maybe, we can introduce some semantic attributes.

For example, we know that both the boy and girl can run. Thus, we can give an high score to the attribute "is able to run". We can think other semantic attributes and in this way we construct the embeddings of the two subjects: the boy and the girl.

\begin{align}q_\text{boy} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{likes beach}, \dots \right]\end{align}

\begin{align}q_\text{girl} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{likes beach}, \dots \right]\end{align}

We can measure the similarity between the boy and girl by doing a dot-product between the two representations:

\begin{align}\text{Similarity}(\text{boy}, \text{girl}) = q_\text{boy} \cdot q_\text{girl}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{boy}, \text{girl}) = \frac{q_\text{boy} \cdot q_\text{girl}}
   {\| q_\text{boy} \| \| q_\text{girl} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


We let the network learn this semantic attributes, by definining them as parameters. We will have some *latent semantic attributes* that the network can, in principle, learn. Note that the word embeddings will probably not be interpretable.

In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**.

## Word Embeddings in Pytorch

Similarly to what we do with the one-hot-encoding representation, we have to define an index for each word in the vocabulary. These will be the keys in a lookup table. That is embeddings are stored in a $|V| \times D$ matrix, were $D$ is the dimensionality of the embeddings, such that the word assigned inex $i$ has the embedding stored in the row of the matrix of index $i$.

The module that permits you to do the embeddings is torch.nn.Embedding which takes to parameters: the dimensionality of the vocabulary and the dimension of the embeddings.





In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random

import spacy
import nltk
from nltk.corpus import stopwords

import string
import re
import seaborn as sns

ModuleNotFoundError: No module named 'spacy'

In [ ]:
# this is all the necessary code to set the seed
def set_seed(seed : int = 123):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

In [ ]:
set_seed()

In [ ]:
word_to_ix = {"hello": 0, "world": 1}

# rows = n. words in the vocabulary
# cols = dimension of the embedding
embeds = nn.Embedding(2, 5)
# tensor to store the index to access the embedding of a word
word = 'hello'
# word = 'world'
lookup_tensor = torch.tensor([word_to_ix[word]], dtype=torch.long)

word_embed = embeds(lookup_tensor)
print(word_embed)

In [ ]:
# load google drive to see the files in google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/sarafrr/pytorch-tutorial-nlp-public.git

In [ ]:
!rm -r pytorch-tutorial-nlp-public

In [ ]:
# from the folder containing the data, obtain the list of all the files
from glob import glob
file_list = glob( "/content/data/Shakespeare/*.txt")

print(file_list)

# Exercise: Compute the Word Embedding Continuous Bag-of-Words

The Continuous Bag-of-Words (CBOW) is widely used in NLP, it is a way to embed the context of few words before and after of the target word.

CBOW is used to train word embeddings and these embeddings are used as initialisation of a more complicate model.

This is usually referred as *pretraining embeddings*.

Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}\left(A(\sum_{w \in C} q_w) + b\right)\end{align}

where $q_w$ is the embedding for word $w$.


Implement the CBOW class in the following snippet of code and train it.

In [ ]:
# CONSTANTS - do not modify this cell of code

# 2 words to the left and two to the right
CONTEXT_SIZE = 2
EMBEDDING_SIZE = 100
EPOCHS = 50
LEARNING_RATE = 1e-3

In [ ]:
# download stopwords
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
stop_words = set(stopwords.words('english'))# e.g.: a, in, is
print(stop_words)
punctuation = set(string.punctuation)
print(punctuation)

In [ ]:
def prepare_corpus(file_list : list, min_size : int = 1):
  ''' Read and prepare the corpus as a list of sentences '''
  # retrieve all the text from the files
  # a string we named complete_text
  complete_text = ''
  for p in file_list:
    with open(p, 'r') as f:
      lines = f.readlines()
      for l in lines:
        complete_text += l
  # retrieve the sentences by the complete_text
  sentences = re.split('[.!?]', complete_text)

  # filter the sentences by the number of words
  corpus = []
  # sentences is a list of strings
  for s in sentences:
    s = s.replace('\n', ' ')
    if len(s.split()) > min_size*2 + 1:
      corpus.append(s)
  return corpus

corpus = prepare_corpus(file_list=file_list, min_size = 2)
print(corpus)

In [ ]:
def prepare_data(corpus : list, context_size : int):
  ''' Tokenizes the data, removing the stop words and the punctuation.
      Creates and returns data composed by the tuple of the context and the target,
      and the dictionaries
  '''
  voc = set()
  tokenized_corpus = []
  for i in corpus:
    tokenized = nltk.word_tokenize(i)
    # make capital letters lowercase
    tokenized = [x.lower() for x in tokenized]
    # remove stopwords and punctuation
    tokenized = [x for x in tokenized if x not in (stop_words|punctuation)]
    tokenized_corpus.append(tokenized)

    voc |= set(tokenized)

  w_to_i = dict(zip(voc,range(len(voc))))
  i_to_w = dict(zip(range(len(voc)),voc))

  # create the data
  data = []
  for s in tokenized_corpus:
    for i in range(context_size, len(s)-context_size):
      context_idx = [w_to_i[s[i-j-1]] for j in range(context_size)]+ \
              [w_to_i[s[i+j+1]] for j in range(context_size)]
      target_idx = w_to_i[s[i]]
      data.append((context_idx, target_idx))

  return data, w_to_i, i_to_w

# Question 1

Implement CBOW model by defining an appropriate class which extends `nn.Module` and train it on the data.

In [ ]:
# Create the model
class CBOW(nn.Module):

    def __init__(self, ''' COMPLETE HERE '''):
        super(CBOW, self).__init__()
        ''' COMPLETE HERE '''

    def forward(self, x):
        ''' COMPLETE HERE '''
        return x

# Question 2
Try to do the appropriate modifications to this notebook to use the GPU. (Tip: see the introductory notebooks about PyTorch, specifically 2_Introduction_to_Pytorch.ipynb or see the notebook NLP_homework1_skipgram(hw).ipynb.)

# Question 3
Plot one sample of the dataset: context, target and the respective prediction and see that the prediction is equal to the target. See the last lines of code of the following cell.

In [ ]:
data, w_to_i, i_to_w = prepare_data(corpus, context_size = CONTEXT_SIZE)
print(len(w_to_i))
# implement the loss
# justify the use of this loss
criterion = nn.CrossEntropyLoss()
model = CBOW(''' COMPLETE HERE ''')
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# create the context data and targets in tensors
context_data = torch.tensor([d[0] for d in data])
target_data = torch.tensor([d[1] for d in data])

# create a dataset and a dataloader
dataset = torch.utils.data.TensorDataset(context_data, target_data)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

for e in range(EPOCHS):
  for idx, data in enumerate(dataloader):
    context = data[0]
    target = data[1]
    predicted = model(context)
    loss = criterion(predicted, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f'Epoch {e} - loss: {loss.item()}')

# --- TESTING ---
data, w_to_i, i_to_w = prepare_data(corpus, context_size = CONTEXT_SIZE)
# create the context data and targets in tensors
context_data = torch.tensor([d[0] for d in data])
target_data = torch.tensor([d[1] for d in data])

# create a dataset and a dataloader
dataset = torch.utils.data.TensorDataset(context_data, target_data)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

for idx, data in enumerate(dataloader):

  context = data[0]
  target = data[1]
  out = model(context)
  break

#print result
print(f'Context: {''' COMPLETE HERE '''}')
print(f'Target: {''' COMPLETE HERE '''}')
print(f'Prediction: {''' COMPLETE HERE '''}')


# Question 4 (Updated)
Plot the original embeddings and trained ones and explain the difference between the two.

To do so use the function `tsne = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3)` of the `scikit-learn` package.

Indeed, using T-distributed Stochastic Neighbor Embedding (or other similar functions, such as the PCA), we can see the embeddings in 2D vector space. Explain the results considering the best plot you obtain by one of the two functions. If the plotting is not so explanatory,
feel free to use the cosine similarity or other techniques.

In [ ]:
for i,p in model.named_parameters():
  print(i)

W = ''' COMPLETE HERE '''
print(W.shape)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=3)
W_dec = tsne.fit_transform(W)
x = ''' COMPLETE HERE '''
y = ''' COMPLETE HERE '''

In [ ]:
# check the dimensions
print(W_dec.shape)
print(x.shape)
print(y.shape)
print(len(i_to_w.keys()))

In [ ]:
plot = sns.scatterplot(x=x, y=y)

for i in range(0,W_dec.shape[0]):
     plot.text(x[i], y[i], list(i_to_w.keys())[i], horizontalalignment='center', size='small', color='black', weight='semibold');

Or using the PCA.

In [ ]:
from sklearn import decomposition

orig_model = CBOW(''' COMPLETE HERE ''')
W1_orig = '''COMPLETE HERE '''

svd = decomposition.TruncatedSVD(n_components=2)
W_orig_dec = svd.fit_transform(W1_orig)
x = ''' COMPLETE HERE '''
y = ''' COMPLETE HERE '''
plot = sns.scatterplot(x=x, y=y)

for i in range(0,W_orig_dec.shape[0]):
     plot.text(x[i], y[i], list(i_to_w.keys())[i], horizontalalignment='center', size='small', color='black', weight='semibold');